# Описание проекта «Определение ''токсичных'' комментариев»

**Дано:**
Набор данных (комментарии пользователей к товарам в интеренет-магазине) с разметкой о токсичности: столбец `text` содержит текст комментария, а `toxic` — целевой признак.

**Требуется:**
Модель для классифицирования комментариев на позитивные и негативные со значением метрики качества `F1` не меньше `0.75`

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-инструментария" data-toc-modified-id="Импорт-инструментария-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт инструментария</a></span></li><li><span><a href="#Глобальные-константы" data-toc-modified-id="Глобальные-константы-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Глобальные константы</a></span></li><li><span><a href="#Функции" data-toc-modified-id="Функции-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Функции</a></span></li><li><span><a href="#Обзор-исходных-данных" data-toc-modified-id="Обзор-исходных-данных-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Обзор исходных данных</a></span></li></ul></li><li><span><a href="#Моделирование" data-toc-modified-id="Моделирование-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Моделирование</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Градиентный-бустинг" data-toc-modified-id="Градиентный-бустинг-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Градиентный бустинг</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li></ul></div>

## Подготовка

### Импорт инструментария

In [1]:
import pandas as pd
import nltk
import re
import warnings
warnings.filterwarnings('ignore')

from tqdm                            import tqdm

from pymystem3                       import Mystem

from nltk.corpus                     import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection         import train_test_split
from sklearn.linear_model            import LogisticRegression
from sklearn.tree                    import DecisionTreeClassifier
from sklearn.ensemble                import RandomForestClassifier
from sklearn.ensemble                import GradientBoostingClassifier
from sklearn.model_selection         import cross_val_score
from sklearn.metrics                 import f1_score

### Глобальные константы

In [2]:
RND_ST   = 12345      # значение параметра варианта рандомизации
CV       = 3          # кратность разбиения данных при кросс-валидации

### Функции

Функция для очистки текста от ненужных символов

In [3]:
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split())

Функция для лемматизации текста

In [4]:
def lemmatize(text):
    words = text.lower().split()
    lemm_list = []
    for word in words:
        wnl = nltk.WordNetLemmatizer()
        lemm_word = wnl.lemmatize(word)
        lemm_list.append(lemm_word)
    lemm_text = ' '.join(lemm_list)
      
    return lemm_text

### Обзор исходных данных

In [5]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [6]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [8]:
df.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [9]:
df.duplicated().sum()

0

In [10]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [11]:
df['toxic'].mean()

0.10161213369158527

**Замечания:**
* в исходных данных отсутствуют явные пропуски;
* в исходных данных отсутствуют полные дубликаты;
* по сути задача является задачей бинарной классификации;
* распределение классов в данных неравномерно (примерно `1` к `9`)

# Выводы
* исходные данные качественные;
* общая предъобработка не требуется

## Моделирование

Тренировочная и тестовая выборки

In [12]:
train, test = train_test_split(df,
                               test_size   =0.15,
                               random_state=RND_ST)

print('Доля `токсичных` ответов в общей выборке:         {:.2%}'.format(df   ['toxic'].mean()))
print('Доля `токсичных` ответов в тренировочной выборке: {:.2%}'.format(train['toxic'].mean()))
print('Доля `токсичных` ответов в тестовой выборке:      {:.2%}'.format(test ['toxic'].mean()))

Доля `токсичных` ответов в общей выборке:         10.16%
Доля `токсичных` ответов в тренировочной выборке: 10.15%
Доля `токсичных` ответов в тестовой выборке:      10.24%


In [13]:
corpus_train = train['text'].tolist()
corpus_test  = test ['text'].tolist()

for i in tqdm(range(len(corpus_train))):
    corpus_train[i] = lemmatize(clear_text(corpus_train[i]))
for i in tqdm(range(len(corpus_test ))):
    corpus_test[i]  = lemmatize(clear_text(corpus_test[i] ))

train['lemm_text'] = pd.Series(data =corpus_train,
                               index=train.index)
test ['lemm_text'] = pd.Series(data =corpus_test,
                               index=test .index)

100%|██████████| 23894/23894 [00:05<00:00, 4476.23it/s]


Векторизация текста и выделение признаков

In [14]:
nltk.download('stopwords')
stopwords    = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
features_train = count_tf_idf.fit_transform(train['lemm_text'])
features_test  = count_tf_idf.transform    (test ['lemm_text'])

target_train   = train['toxic']
target_test    = test ['toxic']

In [16]:
features_train.shape

(135398, 143577)

### Логистическая регрессия

In [17]:
f1_lr_train = cross_val_score(estimator  =LogisticRegression(random_state=RND_ST),
                              X          =features_train,
                              y          =target_train,
                              cv         =CV,
                              scoring    ='f1',
                              error_score='raise')

In [18]:
print('Среднее качество модели `Логистической регрессии` на обучающей выборке при кросс-валидации: {:.2%}'
      .format(f1_lr_train.mean())
     )

Среднее качество модели `Логистической регрессии` на обучающей выборке при кросс-валидации: 70.43%


In [19]:
f1_lr_train = cross_val_score(estimator  =LogisticRegression(random_state=RND_ST,
                                                             class_weight='balanced'),
                              X          =features_train,
                              y          =target_train,
                              cv         =CV,
                              scoring    ='f1',
                              error_score='raise')

In [20]:
print('Среднее качество модели `Логистической регрессии` на обучающей выборке с балансировкой классов при кросс-валидации: {:.2%}'
      .format(f1_lr_train.mean())
     )

Среднее качество модели `Логистической регрессии` на обучающей выборке с балансировкой классов при кросс-валидации: 74.71%


### Дерево решений

In [21]:
best_mean_f1_dtc = 0
best_depth_dtc   = 0

In [22]:
for depth in tqdm(range(3, 11)):
    mean_f1 = cross_val_score(estimator  =DecisionTreeClassifier(random_state=RND_ST,
                                                                 max_depth   =depth),
                              X          =features_train,
                              y          =target_train,
                              cv         =CV,
                              scoring    ='f1',
                              error_score='raise'
                             ).mean()
        
    if mean_f1 > best_mean_f1_dtc:
        best_mean_f1_dtc = mean_f1
        best_depth_dtc   = depth

100%|██████████| 8/8 [02:50<00:00, 21.35s/it]


In [23]:
print('----- Лучший результат -----')
print('Среднее значение F1:   {:.2f}'.format(best_mean_f1_dtc))
print('Глубина дерева:        {}'    .format(best_depth_dtc))

----- Лучший результат -----
Среднее значение F1:   0.59
Глубина дерева:        10


### Случайный лес

In [24]:
total_mean_f1_rfc      = 0
total_depth_rfc        = 0
total_n_estimators_rfc = 0

In [25]:
for depth in tqdm(range(3, 11)):
    best_mean_f1      = 0
    best_depth        = 0
    best_n_estimators = 0
        
    for n_estimator in range(2, 5):
        mean_f1 = cross_val_score(estimator  =RandomForestClassifier(random_state=RND_ST, 
                                                                     n_estimators=n_estimator, 
                                                                     max_depth   =depth),
                                  X          =features_train,
                                  y          =target_train,
                                  cv         =CV,
                                  scoring    ='f1',
                                  error_score='raise'
                                 ).mean()
        
        if mean_f1 > best_mean_f1:
            best_mean_f1      = mean_f1
            best_depth        = depth
            best_n_estimators = n_estimator
        
    if best_mean_f1 > total_mean_f1_rfc:
        total_mean_f1_rfc      = best_mean_f1
        total_depth_rfc        = best_depth
        total_n_estimators_rfc = best_n_estimators

100%|██████████| 8/8 [00:29<00:00,  3.71s/it]


In [26]:
print('----- Лучший результат -----')
print('Среднее значение F1:    {:.2f}'.format(total_mean_f1_rfc))
print('Глубина деревьев:       {}'    .format(total_depth_rfc))
print('Кол-во оценщиков:       {}'    .format(total_n_estimators_rfc))

----- Лучший результат -----
Среднее значение F1:    0.04
Глубина деревьев:       10
Кол-во оценщиков:       2


### Градиентный бустинг

In [27]:
total_mean_f1_gbc      = 0
total_depth_gbc        = 0
total_n_estimators_gbc = 0

In [28]:
for depth in tqdm(range(3, 11)):
    best_mean_f1      = 0
    best_depth        = 0
    best_n_estimators = 0
        
    for n_estimator in range(2, 5):
        mean_f1 = cross_val_score(estimator = GradientBoostingClassifier(learning_rate   =0.1,
                                                                         n_estimators    =n_estimator,
                                                                         subsample       =1.0,
                                                                         max_depth       =depth,
                                                                         random_state    =RND_ST,
                                                                         verbose         =0,
                                                                         max_leaf_nodes  =3,
                                                                         n_iter_no_change=5,
                                                                         tol             =0.0001),
                                  X          =features_train,
                                  y          =target_train,
                                  cv         =CV,
                                  scoring    ='f1',
                                  error_score='raise'
                                 ).mean()
        
        if mean_f1 > best_mean_f1:
            best_mean_f1      = mean_f1
            best_depth        = depth
            best_n_estimators = n_estimator
        
    if best_mean_f1 > total_mean_f1_gbc:
        total_mean_f1_gbc      = best_mean_f1
        total_depth_gbc        = best_depth
        total_n_estimators_gbc = best_n_estimators

100%|██████████| 8/8 [46:44<00:00, 350.55s/it]


In [29]:
print('----- Лучший результат -----')
print('Среднее значение F1:    {:.2f}'.format(total_mean_f1_gbc))
print('Глубина деревьев:       {}'    .format(total_depth_gbc))
print('Кол-во оценщиков:       {}'    .format(total_n_estimators_gbc))

----- Лучший результат -----
Среднее значение F1:    0.02
Глубина деревьев:       3
Кол-во оценщиков:       4


## Тестирование

Лучшее по качеству предсказание из рассмотренных моделей показала модель `Логистической регрессии` с балансировкой классов.

In [30]:
predict_lr_test = (LogisticRegression(random_state=RND_ST,
                                      class_weight='balanced')
                   .fit(features_train, target_train)
                   .predict(features_test)
                  )

In [31]:
f1_score(target_test, predict_lr_test)

0.7543859649122807

**Замечания:**
* на тестовой выборке модель `Логистической регрессии` показа результат лучше, чем на тренировочной

# Выводы
* Из рассмотренных моделей наилучшие результаты показывает самая простая модель - модель `Логистической регрессии`
* модель `Логистической регрессии` дает предсказания приемлемого качества